In [1]:
using CSV
using DataFrames
using Plots
using Measures

In [2]:
include("../src/julia/dcopf.jl");

In [3]:
# # Run the model for all years
# scenario = 11
# gen_prop_name = "vivienne_2050"
# bus_prop_name = "vivienne_2050"
# branch_prop_name = "vivienne_2050"
# out_path = "./out/Scenario$(scenario)/";

# for year in 1998:2019
#     println("Running scenario $(scenario) for year $(year)")
#     run_model(scenario, year, gen_prop_name, branch_prop_name, bus_prop_name, out_path)
# end

In [ ]:
# Reproduce figure 1 from paper
scenario = 11
directory_path = "./out/Scenario$(scenario)/"

# Define the years you have run the model for
years = 1998:2019
num_years = length(years)
num_qms = 48  # Number of quarter months in a year

# Read the quarter month to number of days mapping
dayofqm = CSV.read("../data_tmp/qm_to_numdays.csv", DataFrame)
nhours = dayofqm.Days .* 24  # Convert days to hours
cum_nhours = cumsum(nhours)  # Cumulative hours to map time steps

# Directory where the load shedding results are stored
lhscenario = 4  # Adjust this if your scenario number is different

metrics = Dict{String, Matrix}()

# Initialize matrices to hold the results
total_load_shedding = zeros(num_years, num_qms)
max_hourly_load_shedding = zeros(num_years, num_qms)
num_hours_load_shedding = zeros(num_years, num_qms)

# Loop over each year to process load shedding data
for (i, year) in enumerate(years)
    # Construct the file path for the current year's load shedding data
    loadshed_file = "$(directory_path)/load_shedding_$(year).csv"

    # Read the load shedding data for the current year
    loadshed_df = CSV.read(loadshed_file, DataFrame, header=false)
    loadshed = Matrix(loadshed_df)

    # Sum load shedding over all buses for each hour
    total_loadshed_per_hour = sum(loadshed, dims=1)  # Result is 1 x nt

    # Process each quarter month
    for q in 1:num_qms
        # Determine the start and end indices for the current quarter month
        if q == 1
            start_idx = 1
        else
            start_idx = cum_nhours[q-1] + 1
        end
    end_idx = cum_nhours[q]
    
    # Ensure indices are integers
    start_idx = Int(start_idx)
    end_idx = Int(end_idx)
    
    # Extract the load shedding data for the current quarter month
    ls_qm = total_loadshed_per_hour[1, start_idx:end_idx]
    
    # Compute total load shedding for the quarter month
    total_ls = sum(ls_qm)
    total_load_shedding[i, q] = total_ls
    
    # Compute maximum hourly load shedding for the quarter month
    max_ls = maximum(ls_qm)
    max_hourly_load_shedding[i, q] = max_ls
    
    # Compute number of hours with load shedding in the quarter month
    num_hours_ls = count(ls_qm .> 0)
    num_hours_load_shedding[i, q] = num_hours_ls
    end
end

merge!(metrics, Dict(
    "total_load_shedding" => total_load_shedding, 
    "max_hourly_load_shedding" => max_hourly_load_shedding, 
    "num_hours_load_shedding" => num_hours_load_shedding))

In [ ]:
# Prepare labels for the heatmaps
year_labels = string.(years)
qm_labels = 1:4:num_qms

# Set up plotting parameters
default(size=(1600, 400))

# Plot all three heatmaps in a single figure using subplots
plot(
    heatmap(
        metrics["total_load_shedding"]/1000,
        xlabel="Quarter Month",
        ylabel="",
        title="Total Load Shedding (GWh)",
        xticks=(1:4:num_qms, qm_labels),
        yticks=(1:num_years, year_labels),
        cbar_title="",
        yflip=true
    ),
    heatmap(
        metrics["max_hourly_load_shedding"]/1000,
        xlabel="Quarter Month",
        ylabel="",
        title="Maximum Hourly Load Shedding (GW)",
        xticks=(1:4:num_qms, qm_labels),
        yticks=(1:num_years, year_labels),
        cbar_title="",
        yflip=true
    ),
    heatmap(
        metrics["num_hours_load_shedding"],
        xlabel="Quarter Month",
        ylabel="",
        title="Number of Hours with Load Shedding",
        xticks=(1:4:num_qms, qm_labels),
        yticks=(1:num_years, year_labels),
        cbar_title="",
        yflip=true
    ),
    layout=(1, 3),
    bottom_margin=10mm
)
# Save the combined plot
savefig("./load_shedding_heatmap.png");